In [1]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d kazanova/sentiment140

Saving kaggle.json to kaggle.json
 96% 78.0M/80.9M [00:01<00:00, 43.0MB/s]
100% 80.9M/80.9M [00:01<00:00, 57.8MB/s]


In [6]:
import numpy as np
from zipfile  import ZipFile
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
!pip install pactools
from pactools.grid_search import GridSearchCVProgressBar

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


     |████████████████████████████████| 71kB 3.5MB/s 
  Created wheel for pactools: filename=pactools-0.2.0b0-cp36-none-any.whl size=81682 sha256=d03e3872ad62a998c0124ff10b0a0e5a441166d8b5095514925eaf41ebcc84e1
  Stored in directory: /root/.cache/pip/wheels/5b/a6/57/f1df50567735175243e07792baced6076d67ab30ca1c138b71
Successfully built pactools


In [0]:
z=ZipFile("sentiment140.zip","r")
z.extractall()

In [8]:
df=pd.read_csv("/content/training.1600000.processed.noemoticon.csv",encoding='ISO-8859-1',names=["target", "ids", "date", "flag", "user", "text"])
# df.head()
data=df[["target","text"]]
data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [10]:
from tqdm import tqdm
tqdm.pandas()
stop=stopwords.words("english")
punctuation=string.punctuation
stem=PorterStemmer()
clean="@\S+|https?:\S+|http?:\S|"
full_words={"can't":"can not","aren't":"are not","isn't":"is not","that's":"that is","i'm":"i am","it's":"it is"}
def regex_preprocess(text):
    corp=re.sub(clean,"",str(text).lower())
    words=word_tokenize(corp)
    out=""
    for i in words:
        if i not in stop and punctuation:
            out+=" "+stem.stem(i)
    return out
 



# data=data.iloc[:20000,:]
data["text"]=data['text'].progress_apply(lambda x: regex_preprocess(x))
# apply(lambda x: regex_preprocess(x))
print(data.head())

<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: Dep

   target                                               text
0       0   - awww , 's bummer . shoulda got david carr t...
1       0   upset ca n't updat facebook text ... might cr...
2       0   dive mani time ball . manag save 50 % rest go...
3       0                    whole bodi feel itchi like fire
4       0               , 's behav . 'm mad . ? ca n't see .


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
xtrain,xtest,ytrain,ytest=train_test_split(data["text"],data["target"],random_state=0,test_size=0.2)
V=TfidfVectorizer(ngram_range=(1,2),max_features=10000,stop_words='english')
# data["text"]=V.fit_transform(data.text)
# print(data.head())
V.fit(xtrain)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [12]:
xtrain1=V.transform(xtrain)

nb=MultinomialNB()
param={'alpha':[0.001,0.01,0.1,0.3,0.5,1.0,5.0,10.0],'fit_prior':[True,False]}
gnb=GridSearchCV(nb,param,cv=15)

gnb.fit(xtrain1,ytrain)
print(gnb.best_score_)
print(gnb.best_params_)
#0.756


0.7564054703477845
{'alpha': 10.0, 'fit_prior': True}
0.7564054703477845
{'alpha': 10.0, 'fit_prior': True}


In [13]:

param_lr={'penalty':['l1','l2'],'C':[0.2,0.5,0.7,1.0,1.2,1.5,1.8]}
lr=LogisticRegression(random_state=0,n_jobs=-1)
glr=GridSearchCV(lr,param_lr,cv=15,n_jobs=-1)
glr.fit(xtrain1,ytrain)
print(glr.best_score_)
print(glr.best_params_)
#train 0.773
#test 0.774

0.7734234373588086
{'C': 0.5, 'penalty': 'l2'}


In [15]:
lr2=LogisticRegression(random_state=0,n_jobs=-1,C=0.5,penalty='l2')
lr2.fit(xtrain1,ytrain)
# joblib.dump(lr2,'saved_LR')
print(accuracy_score(ytest,lr2.predict(V.transform(xtest))))

0.773878125


In [19]:
def output(text):
    out=regex_preprocess(text)
    result=gnb.predict(V.transform([out]))+lr2.predict(V.transform([out]))
    if result == [0]:
        print(f"{text} ==> Negative sentiment")

    else:
        print(f"{text} ==>Positive sentiment")
output("killing threee man with knife")
output("sick realli cheap hurt much eat real food plu")
output("motivation comes from mind")

killing threee man with knife ==> Negative sentiment
sick realli cheap hurt much eat real food plu ==> Negative sentiment
motivation comes from mind ==>Positive sentiment
